In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [3]:
# Load dataset
df = pd.read_csv('socal2.csv')

In [4]:
df.head()  # Display the first few rows of the dataset

image_id                 street             citi  n_citi  bed  bath  sqft  \
0         0  1317 Van Buren Avenue  Salton City, CA     317    3   2.0  1560   
1         1         124 C Street W      Brawley, CA      48    3   2.0   713   
2         2        2304 Clark Road     Imperial, CA     152    3   1.0   800   
3         3     755 Brawley Avenue      Brawley, CA      48    3   1.0  1082   
4         4  2207 R Carrillo Court     Calexico, CA      55    4   3.0  2547   

    price  
0  201900  
1  228500  
2  273950  
3  350000  
4  385100

In [5]:
df.tail()  # Display the last few rows of the dataset

image_id                     street               citi  n_citi  bed  \
15469     15469    4156 Sterlingview Drive       Moorpark, CA     227    5   
15470     15470         4355 Avenida Prado  Thousand Oaks, CA     372    5   
15471     15471  12717 Koenigstein Rd Road    Santa Paula, CA     338    3   
15472     15472           36 Kunkle Street       Oak View, CA     253    4   
15473     15473        4562 Eastbourne Bay         Oxnard, CA     261    2   

       bath  sqft   price  
15469   4.1  4092  949000  
15470   3.0  2773  949900  
15471   2.0  1576  920000  
15472   2.0  2086  997000  
15473   2.0  1216  975000

In [6]:
df.shape

(15474, 8)

In [7]:
df.describe()

image_id        n_citi           bed          bath          sqft  \
count  15474.000000  15474.000000  15474.000000  15474.000000  15474.000000   
mean    7736.500000    216.597518      3.506398      2.453251   2173.913209   
std     4467.103368    112.372985      1.034838      0.958742   1025.339617   
min        0.000000      0.000000      1.000000      0.000000    280.000000   
25%     3868.250000    119.000000      3.000000      2.000000   1426.000000   
50%     7736.500000    222.500000      3.000000      2.100000   1951.000000   
75%    11604.750000    315.000000      4.000000      3.000000   2737.750000   
max    15473.000000    414.000000     12.000000     36.000000  17667.000000   

              price  
count  1.547400e+04  
mean   7.031209e+05  
std    3.769762e+05  
min    1.950000e+05  
25%    4.450000e+05  
50%    6.390000e+05  
75%    8.349750e+05  
max    2.000000e+06

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15474 entries, 0 to 15473
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   image_id  15474 non-null  int64  
 1   street    15474 non-null  object 
 2   citi      15474 non-null  object 
 3   n_citi    15474 non-null  int64  
 4   bed       15474 non-null  int64  
 5   bath      15474 non-null  float64
 6   sqft      15474 non-null  int64  
 7   price     15474 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 967.3+ KB


In [9]:
df.isnull().sum()  # Check for missing values

image_id    0
street      0
citi        0
n_citi      0
bed         0
bath        0
sqft        0
price       0
dtype: int64

In [10]:
# Check column names
print("Columns:", df.columns.tolist())

Columns: ['image_id', 'street', 'citi', 'n_citi', 'bed', 'bath', 'sqft', 'price']


In [11]:
df.columns = df.columns.str.strip().str.lower()
print("Cleaned Columns:", df.columns.tolist())

Cleaned Columns: ['image_id', 'street', 'citi', 'n_citi', 'bed', 'bath', 'sqft', 'price']


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15474 entries, 0 to 15473
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   image_id  15474 non-null  int64  
 1   street    15474 non-null  object 
 2   citi      15474 non-null  object 
 3   n_citi    15474 non-null  int64  
 4   bed       15474 non-null  int64  
 5   bath      15474 non-null  float64
 6   sqft      15474 non-null  int64  
 7   price     15474 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 967.3+ KB


In [13]:
df['image_path'] = df['image_id'].astype(str).apply(lambda x: f"socal/socal_pics/{x}.jpg")
print(df[['image_id', 'image_path']].head())

   image_id              image_path
0         0  socal/socal_pics/0.jpg
1         1  socal/socal_pics/1.jpg
2         2  socal/socal_pics/2.jpg
3         3  socal/socal_pics/3.jpg
4         4  socal/socal_pics/4.jpg


In [14]:
# Step 1: Extract tabular features
tabular_features = df[['sqft', 'bed', 'bath']].values

In [15]:


# Step 2: Extract image features
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

base_model = MobileNetV2(weights='imagenet', include_top=False)
feature_model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

def extract_image_features(img_path):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        features = feature_model.predict(img_array)
        return features.flatten()
    except:
        return np.zeros(1280)

# Assuming df['image_path'] contains paths to your images
image_features = np.array([extract_image_features(path) for path in df['image_path']])

C:\Users\Muhammad Haseeb\AppData\Local\Temp\ipykernel_5056\2037307631.py:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False)


In [16]:
# Step 3: Combine features
X = np.hstack((image_features, tabular_features))
y = df['price'].values

In [17]:
# # 1. Extract image features (example: assuming you have file paths in df['image_path'])
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import GlobalAveragePooling2D
# import numpy as np

# # Load MobileNetV2 as feature extractor
# base_model = MobileNetV2(weights='imagenet', include_top=False)
# feature_model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

# def extract_image_features(img_path):
#     try:
#         img = image.load_img(img_path, target_size=(224, 224))
#         img_array = image.img_to_array(img)
#         img_array = np.expand_dims(img_array, axis=0)
#         img_array = preprocess_input(img_array)
#         features = feature_model.predict(img_array)
#         return features.flatten()
#     except:
#         return np.zeros(1280)

# # 2. Extract features from all images
# image_features = np.array([extract_image_features(path) for path in df['image_path']])

# # 3. Get tabular features
# tabular_features = df[['sqft', 'bed', 'bath']].values  # shape (N, 3)

# # 4. Combine image + tabular features
# X_combined = np.hstack((image_features, tabular_features))  # shape (N, 1283)
# y = df['price']


In [18]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [19]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

In [20]:
# Train model
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [21]:

# Save model & scaler
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)